# DevInsight Demo Notebook
## Exploring GitHub Analytics with BigQuery and AI

This notebook demonstrates:
- Querying GitHub data from BigQuery
- Analyzing issues, commits, and PRs
- Testing AI-powered insights

In [ ]:
# Install required packages
!pip install google-cloud-bigquery pandas matplotlib seaborn

In [ ]:
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up BigQuery client
project_id = os.getenv('GCP_PROJECT_ID', 'your-project-id')
client = bigquery.Client(project=project_id)
dataset_id = 'github_analytics'

print(f"Connected to project: {project_id}")

## 1. Query Recent Issues

In [ ]:
query = f"""
SELECT 
    number,
    title,
    state,
    created_at,
    labels
FROM `{project_id}.{dataset_id}.issues`
WHERE created_at >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
ORDER BY created_at DESC
LIMIT 20
"""

df_issues = client.query(query).to_dataframe()
df_issues.head()

## 2. Issue State Distribution

In [ ]:
query = f"""
SELECT 
    state,
    COUNT(*) as count
FROM `{project_id}.{dataset_id}.issues`
GROUP BY state
"""

df_states = client.query(query).to_dataframe()

plt.figure(figsize=(8, 6))
plt.pie(df_states['count'], labels=df_states['state'], autopct='%1.1f%%')
plt.title('Issue State Distribution')
plt.show()

## 3. Commit Activity Over Time

In [ ]:
query = f"""
SELECT 
    DATE(author_date) as date,
    COUNT(*) as commits
FROM `{project_id}.{dataset_id}.commits`
WHERE author_date >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
GROUP BY date
ORDER BY date
"""

df_commits = client.query(query).to_dataframe()

plt.figure(figsize=(12, 6))
plt.plot(df_commits['date'], df_commits['commits'], marker='o')
plt.xlabel('Date')
plt.ylabel('Number of Commits')
plt.title('Commit Activity (Last 30 Days)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Test AI Agent

In [ ]:
import sys
sys.path.append('../backend')

from src.ai_agent import AIAgent
from src.elasticsearch_indexer import ElasticsearchIndexer

# Initialize AI agent
agent = AIAgent(project_id=project_id)

# Test query
question = "What are the main issues affecting the project?"
response = agent.answer_query(question)

print(f"Question: {question}")
print(f"\nAnswer: {response['answer']}")
print(f"\nConfidence: {response['confidence']}")